In [1]:
%load_ext cython

In [2]:
import numpy as np
#cimport numpy as np
from contract import ContractTensorNetwork

In [3]:
def KraussToTheta(np.ndarray[np.complex128, ndim=3] kraus):
    # Convert from the Kraus representation to the "Theta" representation.
    # The "Theta" matrix T of a CPTP map whose chi-matrix is X is defined as:
    # T_ij = \sum_(ij) [ X_ij (P_i o (P_j)^T) ]
    # Note that the chi matrix X can be defined using the Kraus matrices {K_k} in the following way
    # X_ij = \sum_k [ <P_i|K_k><K_k|P_j> ]
    # 	   = \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)]
    # So we find that
    # T = \sum_(ij) [ \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)] ] (P_i o (P_j)^T) ]
    # We will store T as a Tensor with dimension = (2 * number of qubits) and bond dimension = 4.
    cdef np.ndarray[np.complex128, ndim=3] Paulis = np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]], dtype=nb.complex128)

    cdef int nq = int(np.log2(kraus.shape[1]))

    cdef double transpose_sign = 0	
    cdef np.ndarray[np.complex128, ndim=2] chi = np.zeros((4**nq, 4**nq), dtype = np.complex128)

    cdef np.ndarray[np.complex128, ndim=4 * nq] theta = np.zeros(tuple([2, 2, 2, 2]*nq), dtype = np.complex128)
    cdef np.ndarray[np.complex128, ndim=4 * nq] PjoPi = np.zeros(tuple([2, 2, 2, 2]*nq), dtype = np.complex128)
    cdef np.ndarray[np.complex128, ndim=4 * nq] PjToPi = np.zeros(tuple([2, 2, 2, 2]*nq), dtype = np.complex128)

    cdef np.ndarray[np.int, ndim=1] supp_Kdag = np.arange(nq, dtype = np.int)
    cdef np.ndarray[np.int, ndim=1] supp_K = np.arange(nq, dtype = np.int) + nq
    cdef np.complex128_t tr_Pi_K = 0
    cdef np.complex128_t tr_Pi_K_dag = 0

    # loop variables.
    cdef int i, j, k, q, index, y_count

    # Preparing the Pauli operators.
    # click = timer()
    cdef np.ndarray[np.int, ndim=2] pauli_operators = np.zeros((4**nq, nq), dtype = np.int)	
    cdef np.ndarray[np.int, ndim=1] transpose_signs = 1
    for i in range(4**nq):
        index = i
        y_count = 0
        for q in range(nq):
            pauli_operators[i, nq - q - 1] = index % 4
            index = int(index//4)
            # Count the number of Ys' for computing the transpose.
            if (pauli_operators[i, q] == 2):
                y_count += 1
        transpose_signs[i] = (-1) ** y_count 

    # click = timer()
    for i in range(4**nq):
        Pi_tensor = [((q,), Paulis[pauli_operators[i, q], :, :]) for q in range(nq)]

        for j in range(4**nq):
            Pj_tensor = [((q,), Paulis[pauli_operators[j, q], :, :]) for q in range(nq)]
            #PjT_tensor = [(tuple(list(range(nq))), transpose_sign * pauli_tensors[j])]

            # click = timer()

            if (i <= j):
                for k in range(kraus.shape[0]):
                    K = [(supp_K, np.reshape(kraus[k, :, :], tuple([2, 2]*nq)))]
                    Kdag = [(supp_Kdag, np.reshape(np.conj(kraus[k, :, :].T), tuple([2, 2]*nq)))]

                    (__, tr_Pi_K) = ContractTensorNetwork(K + Pi_tensor, end_trace=1)
                    (__, tr_Pj_Kdag) = ContractTensorNetwork(Kdag + Pj_tensor, end_trace=1)

                    chi[i, j] += tr_Pi_K * tr_Pj_Kdag

                chi[i, j] /= 4**nq

            else:
                chi[i, j] = np.conj(chi[j, i])

            if (i == j):
                if ((np.real(chi[i, j]) <= -1E-14) or (np.abs(np.imag(chi[i, j])) >= 1E-14)):
                    print("Error: Chi[%d, %d] = %g + i %g" % (i, j, np.real(chi[i, j]), np.imag(chi[i, j])))
                    exit(0)

            (__, PjoPi) = ContractTensorNetwork(Pj_tensor + Pi_tensor, end_trace=0)
            PjToPi = PjoPi * transpose_signs[j]
            theta += chi[i, j] * PjToPi

            # print("Chi[%d, %d] = %g + i %g was computed in %d seconds." % (i, j, np.real(chi[i, j]), np.imag(chi[i, j]), timer() - click))
        # print("----")

    # print("Theta matrix was computed in {} seconds.".format(timer() - click))
    return theta

SyntaxError: invalid syntax (<ipython-input-3-e02f0253786c>, line 1)